<a href="https://colab.research.google.com/github/neo4j-partners/hands-on-lab-neo4j-and-vertex-ai/blob/main/Lab%206%20-%20Vertex%20AI/vertex_ai_embedding.ipynb" target="_blank">
  <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
</a>

# Install Additional Packages
First off, you'll also need to install a few packages.

In [1]:
%pip install --quiet google.cloud.aiplatform

     |████████████████████████████████| 1.8 MB 14.4 MB/s 
     |████████████████████████████████| 230 kB 40.6 MB/s 
     |████████████████████████████████| 107 kB 50.5 MB/s 
     |████████████████████████████████| 46 kB 818 kB/s 
     |████████████████████████████████| 107 kB 58.9 MB/s 
     |████████████████████████████████| 107 kB 64.3 MB/s 
     |████████████████████████████████| 106 kB 67.7 MB/s 
     |████████████████████████████████| 106 kB 65.4 MB/s 
     |████████████████████████████████| 106 kB 59.2 MB/s 
     |████████████████████████████████| 106 kB 72.2 MB/s 
     |████████████████████████████████| 105 kB 61.8 MB/s 
     |████████████████████████████████| 105 kB 71.2 MB/s 
     |████████████████████████████████| 105 kB 54.5 MB/s 
     |████████████████████████████████| 105 kB 61.3 MB/s 
     |████████████████████████████████| 105 kB 69.2 MB/s 
     |████████████████████████████████| 104 kB 61.8 MB/s 
     |████████████████████████████████| 104 kB 63.4 MB/s 
     |██████████

# Restart the Kernel
After you install the additional packages, you need to restart the notebook kernel so it can find the packages.  When you run this, you may get a notification that the kernel crashed.  You can disregard that.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'restart': True, 'status': 'ok'}

# Authenticate your Google Cloud Account
These steps will authenticate the notebook using your Google Cloud credentials.

In [1]:
# Edit these variables!
#PROJECT_ID = 'YOUR-PROJECT-ID'
#STORAGE_BUCKET = 'NAME-OF-BUCKET-FROM-PREVIOUS-LAB'

PROJECT_ID = 'useful-patrol-352218'
STORAGE_BUCKET = 'form13sdjfoiergeoirj'


# You can leave these defaults
REGION = 'us-central1'

In [2]:
import os
os.environ["GCLOUD_PROJECT"] = PROJECT_ID

In [3]:
try:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
except:
    pass

# Train a Model on GCP
We'll use the original and engineered features to train an AutoML model.

In [4]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

dataset = aiplatform.TabularDataset.create(
    display_name="form13-embedding",
    gcs_source=os.path.join("gs://", STORAGE_BUCKET, 'form13', 'embedding.csv'),
)
dataset.wait()

print(f'\tDataset: "{dataset.display_name}"')
print(f'\tname: "{dataset.resource_name}"')

Creating TabularDataset
Create TabularDataset backing LRO: projects/701617915854/locations/us-central1/datasets/4986852579964092416/operations/2931286523498201088
TabularDataset created. Resource name: projects/701617915854/locations/us-central1/datasets/4986852579964092416
To use this TabularDataset in another session:
ds = aiplatform.TabularDataset('projects/701617915854/locations/us-central1/datasets/4986852579964092416')
	Dataset: "form13-embedding"
	name: "projects/701617915854/locations/us-central1/datasets/4986852579964092416"


In [5]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name='form13-embedding',
    optimization_prediction_type='classification'
)

In [ ]:
model = job.run(
    dataset=dataset,
    target_column='target',
    predefined_split_column_name='split',
    model_display_name='form13-embedding',
    disable_early_stopping=False,
    budget_milli_node_hours=1000,
)

No column transformations provided, so now retrieving columns from dataset in order to set default column transformations.
The column transformation of type 'auto' was set for the following columns: ['embedding_0', 'embedding_6', 'embedding_13', 'embedding_12', 'cusip', 'embedding_9', 'split', 'nameOfIssuer', 'embedding_8', 'embedding_14', 'embedding_2', 'embedding_4', 'filingManager', 'embedding_1', 'embedding_3', 'embedding_15', 'shares', 'embedding_7', 'embedding_10', 'value', 'embedding_5', 'embedding_11'].
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7626942176219365376?project=701617915854
AutoMLTabularTrainingJob projects/701617915854/locations/us-central1/trainingPipelines/7626942176219365376 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/701617915854/locations/us-central1/trainingPipelines/7626942176219365376 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects

1000 milli node hours, or one node hour, is the minimum budget that Vertex AI allows.  However, Vertex AI isn't respecting that budget currently.  This job will probably run for two and a half hours.  

We're going to move on while that runs.  You can check on the job later in the [Google Cloud Console](https://console.cloud.google.com/) to see the results.  There's a link to the specific job in the output of the cell above.